In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import kss
from konlpy.tag import Okt  
from konlpy.tag import Kkma 

In [3]:
import time
from tqdm import notebook

In [4]:
import re

In [5]:
okt = Okt()

In [6]:
naver = pd.read_csv("naver_review.csv")
naver

,year,movie_id,reviews
0,2003,1%의 어떤 것,"살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고,..."
1,2003,25시,"(예전리뷰) ""25시"" 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동..."
2,2003,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 SWAT를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너...
3,2003,S.W.A.T. 특수기동대,"수많은 액션영화에는 나름대로의 `공식`이 있습니다. 첫번째, 사격 또는 무술에 있어..."
4,2003,S.W.A.T. 특수기동대,총만 더럽게 쏴되는군.. 마지막에 격투씬 ㅎㅎ 말도 안되는 격투신도 나오고... 해...
...,...,...,...
20956,2021,"사랑하고 사랑받고, 차고 차이고","애니 [사랑하고, 사랑받고, 차고, 차이고] 국내 메인 예고편(한글) : 2021...."
20957,2021,"사랑하고 사랑받고, 차고 차이고","사랑하고 사랑받고, 차고 차이고감독쿠로야나기 토시마사출연한 메구미, 시마자키 노부나..."
20958,2021,"사랑하고 사랑받고, 차고 차이고","사랑하고 사랑받고, 차고 차이고감독쿠로야나기 토시마사출연한 메구미, 시마자키 노부나..."
20959,2021,사일런스 앤드 다크니스,"영화 [사일런스 앤드 다크니스] 메인 예고편 : 2021.05 : 시각 장애, 청각..."


In [12]:
naver['reviews']= naver['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

<ipython-input-12-ea4d2c91792c>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  naver['reviews']= naver['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [13]:
naver

,year,movie_id,reviews
0,2003,1%의 어떤 것,살짝 억지스럽고 너무나 심하게 건전한 점에서 조금 깍였지만너무 보기 좋은 드라마고 ...
1,2003,25시,예전리뷰 시 척박한 현실을 여과없이 드러내는 뛰어난 리얼리즘영화인동시에 진짜와...
2,2003,S.W.A.T. 특수기동대,개봉일에 맞춰 기다리던 를 보았는데 이젠 나아지겠지하고 인내를 하고보다가 너무 화가...
3,2003,S.W.A.T. 특수기동대,수많은 액션영화에는 나름대로의 공식이 있습니다 첫번째 사격 또는 무술에 있어 타의 ...
4,2003,S.W.A.T. 특수기동대,총만 더럽게 쏴되는군 마지막에 격투씬 ㅎㅎ 말도 안되는 격투신도 나오고 해피엔딩이네...
...,...,...,...
20956,2021,"사랑하고 사랑받고, 차고 차이고",애니 사랑하고 사랑받고 차고 차이고 국내 메인 예고편한글 한 메구미순정만화 같...
20957,2021,"사랑하고 사랑받고, 차고 차이고",사랑하고 사랑받고 차고 차이고감독쿠로야나기 토시마사출연한 메구미 시마자키 노부나가 ...
20958,2021,"사랑하고 사랑받고, 차고 차이고",사랑하고 사랑받고 차고 차이고감독쿠로야나기 토시마사출연한 메구미 시마자키 노부나가 ...
20959,2021,사일런스 앤드 다크니스,영화 사일런스 앤드 다크니스 메인 예고편 시각 장애 청각 장애가 있는 딸들시각...


In [14]:
df = pd.read_csv("stopwords-ko.txt")
stopwords = df.iloc[:,0].unique()

In [21]:
naver['word'] = 0

In [93]:
#for i in notebook.tqdm(range(18999,len(naver))):
#    words = naver['reviews'][i]
#    letters_only = re.sub('[^0-9가-힣\s\.\!\?]*', '', str(words))
#    sampledf = pd.DataFrame(okt.pos(letters_only, norm=True, stem=True))
#    try:
#        sampledf = sampledf[(sampledf[1]=='Noun')|(sampledf[1]=='Adjective')|(sampledf[1]=='Verb')]
#        spword = []
#        for word in sampledf[0]:
#            if len(word)>=2:
#                if word not in (stopwords): # 스탑워즈 제외하고 출력하기
#                    spword.append(word)
#        a = pd.DataFrame(spword)[0].unique()
#        naver['word'][i] = a
#    except KeyError:
#        naver['word'][i] = ""

  0%|          | 0/1962 [00:00<?, ?it/s]

<ipython-input-93-0f1eb594478b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_re['word'][i] = a


In [68]:
naver.loc[5999]

year                                                     2008
movie_id                                                   바디
reviews       앞뒤 하나도 안맞고  뒤죽박죽에  개의 조각이 어쩌고 하더니만  개안되는 각각의...
word        [앞뒤, 맞다, 뒤죽박죽, 조각, 어쩌, 하다, 안되다, 스토리, 연결, 되다, 말...
Name: 5999, dtype: object

In [95]:
#naver.to_csv('naver_re.csv',encoding='utf-8-sig')

In [98]:
naver_re = pd.read_csv('naver_re.csv')

In [87]:
naver_re['word'][10900]

"['뭔가' '색다르다' '흥미롭다' '전개' '음뭐' '딱하다' '비평' '싶다' '않다' '하다' '기분' '초반' '중반'\n '아무' '없다' '보다' '마지막' '부분' '가지' '정도' '들다' '무쿠' '재물' '바치다' '이유' '살짝' '나오다'\n '내용' '아마' '지다' '마누라' '살리다' '가족' '살다' '때문' '어리다' '알다' '이상하다' '생기다' '괴물'\n '헐다' '분만' '이름' '까먹다' '딴사람' '생각' '천뢰' '그거' '걸다' '나루토' '나선' '에이' '새벽' '머리'\n '복잡하다' '갈구다' '분명' '분위기' '정말' '초롱초롱' '애절하다' '눈빛' '미안하다' '바라보다' '쿠도' '초점'\n '이해' '왜죽' '그것' '제츠' '약속' '지키다' '이서' '어쩌' '다시' '상자' '들어가다' '맞다' '또래' '많다'\n '대사' '전멸' '도대체' '완벽하다' '들러리' '역할' '수행' '송충이' '멋지다' '기술' '날리다' '명도' '외치다'\n '재미' '아니다' '재밌다' '만족']"

In [102]:
naver_re[naver_re['movie_id'] == "이터널 선샤인"]

,Unnamed: 0,Unnamed: 0.1,year,movie_id,reviews,word
2767,2767,2767,2005,이터널 선샤인,과연 그럴까 망각한 자는 정말로 복이 있는 자일까 사람은 누구나 기억을 안고 ...,['그렇다' '망각' '자다' '이다' '사람' '기억' '안고' '산다' '그것'...
2768,2768,2768,2005,이터널 선샤인,제목 이터널 선샤인 감독 미셸 공드리출연 짐 캐리 케이트 윈슬...,['제목' '터널' '선샤인' '감독' '미셸' '드리다' '출연' '캐리' '케이...
2769,2769,2769,2005,이터널 선샤인,이터널 션샤인 감독미셸 공드리 주연짐 캐리 조엘 역케이트 윈슬렛 ...,['터널' '샤인' '감독' '미셸' '드리다' '주연' '캐리' '조엘' '케이트...
2770,2770,2770,2005,이터널 선샤인,NaN,NaN
2771,2771,2771,2005,이터널 선샤인,누구나 한 번쯤은 생각해 봤을 것이다 이 순간은 정말 잊고 싶다 이 일은 정말 잊...,['하다' '생각' '보다' '순간' '정말' '잊다' '싶다' '사람' '돌아보다...
2772,2772,2772,2005,이터널 선샤인,영화를 아직 안보신분들은 제 글을 읽지 말아 주세요 스포일러 입니다 제 해석과 조금...,['영화' '안보' '신분' '읽다' '줄다' '스포일러' '이다' '해석' '달라...
2773,2773,2773,2005,이터널 선샤인,나는 사랑했었다 스무살도 되지 않은 어린 나 였고내 마음을 전할 능력도 없...,['사랑' '하다' '스무살' '되다' '않다' '어리다' '이다' '마음' '능력...
2774,2774,2774,2005,이터널 선샤인,잠에서 깨어 눈물이 멈추지않던 시절이 있었다이성이 잠드는 순간내 감성은 미친듯이...,['깨다' '눈물' '멈추다' '않다' '시절' '이성' '잠들다' '순간' '감성...
2775,2775,2775,2005,이터널 선샤인,사랑은 아름답지않은데 일상이니까지우고 싶은 기억이기도 한 이별한 사랑지우기 싫은 기...,['사랑' '아름답다' '않다' '일상' '지우다' '싶다' '기억' '하다' '이...
2776,2776,2776,2005,이터널 선샤인,강하고 화려한 장면은 없었지만 이터널 선샤인은 따뜻했다 아참서로에게만 불러주는...,['강하다' '화려하다' '장면' '없다' '터널' '선샤인' '따뜻하다' '차다'...
